# PostgreSQL query example using vanilla driver
 This notebook implements a tranformation of `ResultSet` to `krangl` dataframe for data visualization.

If you use local postgres database, you can run the following container
```bash
docker compose up -d postgres
```

## Install dependencies

In [17]:
// load version variables
%use @file[resources/version.json](currentDir=".")

In [18]:
USE {
    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("org.postgresql:postgresql:$postgresVersion")
        implementation("com.github.holgerbrandl:krangl:$kranglVersion")
    }
}

// list the library, if the library is not exist, restart kernel
notebook.currentClasspath.joinToString("\n")

/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-356/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-stdlib-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-356/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-reflect-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-356/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlinx-serialization-core-jvm-1.6.3.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-356/kotlin-jupyter-script-classpath-shadowed-zip_extracted/annotations-13.0.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-356/kotlin-jupyter-sc

## Load PostgreSQL connection variables
Rename `postgres.example.json` to `postgres.secret.json`. This will create `uri`, `username`, `password` variables.

In [19]:
// Load environment variables
%use @file[resources/postgres.secret.json](currentDir=".")

## Create Postgresql connection

In [20]:
import java.sql.DriverManager
import java.sql.Connection
import java.sql.PreparedStatement

Class.forName("org.postgresql.Driver")

val conn = DriverManager.getConnection(uri, username, password)

conn.prepareStatement("SELECT 1;").execute()

true

## Create extension function to visualize SQL result using Dataframe

In [21]:
fun Connection.executeUpdate(sql: String, init: (PreparedStatement) -> Unit = {}): Int {
    return prepareStatement(sql).use { statement ->
        init(statement)
        statement.executeUpdate()
    }
}

// generically tranform ResultSet to DataFrame
fun Connection.executeQueryToDf(sql: String, init: (PreparedStatement) -> Unit = {}): DataFrame {
    return prepareStatement(sql).use { statement ->
        init(statement)
        val resultSet = statement.executeQuery()
        val headers = mutableListOf<String>()
        for(i in 1..resultSet.metaData.columnCount){
            headers.add(resultSet.metaData.getColumnLabel(i))
        }
        val rows = mutableListOf<String>()
        while(resultSet.next()) {
            for(i in 1..resultSet.metaData.columnCount){
                // TODO: preserve data type in DataFrame
                rows.add(resultSet.getString(i))
            }
        }
        dataFrameOf(*headers.toTypedArray())(*rows.toTypedArray())
    }
}

conn.executeQueryToDf("SELECT 1 as HEALTHCHECK;")

healthcheck
1


## Create User table

In [22]:
val sql = """
    CREATE TABLE IF NOT EXISTS users (
        id SERIAL PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL
    )
""".trimIndent()

conn.executeUpdate(sql)

0

## Insert data to User table

In [23]:
val sql = "INSERT INTO users (name, email) VALUES (?, ?) RETURNING *"

conn.executeQueryToDf(sql) { statement ->
    statement.setString(1, "user4")
    statement.setString(2, "user4@example.com")
}

id,name,email
7,user4,user4@example.com


## Select users

In [24]:
val sql = "SELECT * from users"

conn.executeQueryToDf(sql)


id,name,email
1,user4,user4@example.com
2,user4,user4@example.com
3,user4,user4@example.com
4,user4,user4@example.com
5,user4,user4@example.com
6,user4,user4@example.com
7,user4,user4@example.com
